### District Summary

In [2]:
#dependencies
import pandas as pd
import pathlib

In [3]:
# setting filepaths using pathlib to access csvs 
school_filepath = pathlib.Path("Resources", "schools_complete.csv")
student_filepath = pathlib.Path("Resources", "students_complete.csv")

In [4]:
#reading csvs into data frames
school_data = pd.read_csv(school_filepath)
student_data = pd.read_csv(student_filepath)

In [5]:
#merging data frames into new df
school_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [6]:
# calculating total number of schools - creating new df that takes in only list of schools
school_list_df = school_complete_df[["school_name", "budget"]]

# creating new list of unique schools
unique_school_list = school_list_df["school_name"].unique()

#finding total number of unique high schools
number_of_schools = len(unique_school_list)

In [7]:
# calculating total number of students

student_list_df = school_complete_df[["student_name", "math_score", "reading_score"]]
number_of_students = len(student_list_df)

In [8]:
# calculate total budget
unique_budget_list_df = school_list_df["budget"].unique()

total_budget = unique_budget_list_df.sum()

In [9]:
# calculate average math score
math_score_average = student_list_df["math_score"].mean()

In [10]:
# calculate average reading score
reading_score_average = student_list_df["reading_score"].mean()

In [11]:
# Calculate the percentage of students with a grade >70
number_of_math_scores = len(student_list_df["math_score"])
is_passing_math = student_list_df["math_score"] >= 70

only_passing_math = student_list_df.loc[
    is_passing_math,
    :]

# percent passing math = number of passing scores divided by total number of scores times 100
percent_passing_math = (len(only_passing_math) / number_of_math_scores) * 100

In [12]:
# Calculate the percentage of students with a passing reading score (70 or greater)
number_of_reading_scores = len(student_list_df["reading_score"])
is_passing_reading = student_list_df["reading_score"] >= 70

only_passing_reading = student_list_df.loc[
    is_passing_reading,
    :]

percent_passing_reading = (len(only_passing_reading) / number_of_reading_scores) * 100

In [13]:
# confirm that the number of math and reading scores are equal

assert number_of_math_scores == number_of_reading_scores

In [14]:
#conditional for reading scores equal to or greater than 70
is_passing_both = ((student_list_df["reading_score"] >= 70) & (student_list_df["math_score"] >= 70))

only_passing_both = student_list_df.loc[
    is_passing_both,
    :]

only_passing_both

overall_passing = (len(only_passing_both)/number_of_reading_scores) * 100

In [15]:
# creating summary table
district_summary_df = pd.DataFrame({
    "Total Schools": [number_of_schools],
    "Total Students": [number_of_students],
    "Total Budget": [total_budget],
    "Average Math Score": [math_score_average],
    "Average Reading Score": [reading_score_average],
    "% Passing Math": [percent_passing_math],
    "% Passing Reading": [percent_passing_reading],
    "% Overall Passing": [overall_passing],
    
})

# formatting for style
district_summary_df["Total Students"] = (district_summary_df["Total Students"].map("{:,}".format))
district_summary_df["Total Budget"] = (district_summary_df["Total Budget"].astype(float).map("${:,.2f}".format))
district_summary_df["% Passing Math"] = (district_summary_df["% Passing Math"].astype(float).map("{0:.2f}%".format))
district_summary_df["% Passing Reading"] = (district_summary_df["% Passing Reading"].astype(float).map("{0:.2f}%".format))
district_summary_df["% Overall Passing"] = (district_summary_df["% Overall Passing"].astype(float).map("{0:.2f}%".format))

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## School Summary

In [16]:
# creating a group of schools and setting the index to the school name
school_groups = school_complete_df.groupby(["school_name"], as_index=True)

In [17]:
# pulling the total number of schools per school
total_school_students = school_groups["student_name"].count()

In [18]:
# .first method pulls the first object in a grouping
school_type = school_groups["type"].first()

In [19]:
# if adding, this would sum duplicate values
total_school_budget = school_groups["budget"].first()

In [20]:
per_student_budget = total_school_budget / total_school_students

In [21]:
average_school_math_score = school_groups["math_score"].mean()

In [22]:
average_school_reading_score = school_groups["reading_score"].mean()

In [23]:
passing_math_per_school = school_complete_df[school_complete_df["math_score"]>=70].groupby(['school_name']).size()

In [24]:
pct_passing_math = (passing_math_per_school / total_school_students) * 100

In [25]:
passing_reading_per_school = school_complete_df[school_complete_df["reading_score"]>=70].groupby(['school_name']).size()

pct_passing_reading_per_school = (passing_reading_per_school / total_school_students) * 100

In [26]:
overall_passing_per_school = school_complete_df[(school_complete_df["reading_score"]>=70) & (school_complete_df["math_score"]>=70)].groupby(['school_name']).size()

pct_overall_passing_per_school = (overall_passing_per_school / total_school_students) * 100

In [58]:
# creating summary table for schools
school_summary_df = pd.DataFrame({
    "Total Students Per School": total_school_students,
    "School Type": school_type,
    "Total Budget Per School": total_school_budget,
    "Per Student Budget": per_student_budget,
    "Average Math Score Per School": average_school_math_score,
    "Average Reading Score Per School": average_school_reading_score,
    "% Math Passing Per School": pct_passing_math,
    "% Reading Passing Per School": pct_passing_reading_per_school,
    "% Overall Passing Per School": pct_overall_passing_per_school
})

school_summary_df.index.rename("School Names", inplace=True)

# to run binned df below, the formatting from the school summary df had to be changed to an object, otherwise an object can't be binned using ints
# school_summary_df["Total Students Per School"] = (school_summary_df["Total Students Per School"].map("{:,}".format))
school_summary_df["Total Budget Per School"] = (school_summary_df["Total Budget Per School"].astype(float).map("${:,.2f}".format))
# school_summary_df["% Math Passing Per School"] = (school_summary_df["% Math Passing Per School"].astype(float).map("{0:.2f}%".format))
# school_summary_df["% Reading Passing Per School"] = (school_summary_df["% Reading Passing Per School"].astype(float).map("{0:.2f}%".format))
# school_summary_df["% Overall Passing Per School"] = (school_summary_df["% Overall Passing Per School"].astype(float).map("{0:.2f}%".format))


school_summary_df

,Total Students Per School,School Type,Total Budget Per School,Per Student Budget,Average Math Score Per School,Average Reading Score Per School,% Math Passing Per School,% Reading Passing Per School,% Overall Passing Per School
School Names,,,,,,,,,
Bailey High School,4976,District,"$3,124,928.00",628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,1858,Charter,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,2949,District,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,2739,District,"$1,763,916.00",644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,1468,Charter,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,4635,District,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,427,Charter,"$248,087.00",581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,2917,District,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,4761,District,"$3,094,650.00",650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

In [28]:
top_schools_df = school_summary_df.sort_values(by="% Overall Passing Per School", ascending=False)

top_schools_df.head(5)

,Total Students Per School,School Type,Total Budget Per School,Per Student Budget,Average Math Score Per School,Average Reading Score Per School,% Math Passing Per School,% Reading Passing Per School,% Overall Passing Per School
School Names,,,,,,,,,
Cabrera High School,1858,Charter,"$1,081,356.00",582.0,83.061895,83.975780,94.13%,97.04%,91.33%
Thomas High School,1635,Charter,"$1,043,130.00",638.0,83.418349,83.848930,93.27%,97.31%,90.95%
Griffin High School,1468,Charter,"$917,500.00",625.0,83.351499,83.816757,93.39%,97.14%,90.60%
Wilson High School,2283,Charter,"$1,319,574.00",578.0,83.274201,83.989488,93.87%,96.54%,90.58%
Pena High School,962,Charter,"$585,858.00",609.0,83.839917,84.044699,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [29]:
lowest_schools_df = school_summary_df.sort_values(by="% Overall Passing Per School", ascending=True)

lowest_schools_df.head(5)

,Total Students Per School,School Type,Total Budget Per School,Per Student Budget,Average Math Score Per School,Average Reading Score Per School,% Math Passing Per School,% Reading Passing Per School,% Overall Passing Per School
School Names,,,,,,,,,
Rodriguez High School,3999,District,"$2,547,363.00",637.0,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,2949,District,"$1,884,411.00",639.0,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,2917,District,"$1,910,635.00",655.0,76.629414,81.182722,65.68%,81.32%,53.51%
Hernandez High School,4635,District,"$3,022,020.00",652.0,77.289752,80.934412,66.75%,80.86%,53.53%
Johnson High School,4761,District,"$3,094,650.00",650.0,77.072464,80.966394,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [30]:
is_ninth_grade = school_complete_df["grade"] == "9th"
ninth_grade_math = school_complete_df.loc[is_ninth_grade,:].groupby("school_name")["math_score"].mean()

In [31]:
is_tenth_grade = school_complete_df["grade"] == "10th"
tenth_grade_math = school_complete_df.loc[is_tenth_grade,:].groupby("school_name")["math_score"].mean()

In [32]:
is_eleventh_grade = school_complete_df["grade"] == "10th"
eleventh_grade_math = school_complete_df.loc[is_eleventh_grade,:].groupby("school_name")["math_score"].mean()

In [33]:
is_twelfth_grade = school_complete_df["grade"] == "10th"
twelfth_grade_math = school_complete_df.loc[is_twelfth_grade,:].groupby("school_name")["math_score"].mean()

In [34]:
grade_math_summary_df = pd.DataFrame({
    "9th Grade": ninth_grade_math,
    "10th Grade": tenth_grade_math,
    "11th Grade": eleventh_grade_math,
    "12th Grade": twelfth_grade_math
})

grade_math_summary_df.index.rename("School Names", inplace=True)


grade_math_summary_df

,9th Grade,10th Grade,11th Grade,12th Grade
School Names,,,,
Bailey High School,77.083676,76.996772,76.996772,76.996772
Cabrera High School,83.094697,83.154506,83.154506,83.154506
Figueroa High School,76.403037,76.539974,76.539974,76.539974
Ford High School,77.361345,77.672316,77.672316,77.672316
Griffin High School,82.044010,84.229064,84.229064,84.229064
Hernandez High School,77.438495,77.337408,77.337408,77.337408
Holden High School,83.787402,83.429825,83.429825,83.429825
Huang High School,77.027251,75.908735,75.908735,75.908735
Johnson High School,77.187857,76.691117,76.691117,76.691117


## Reading Score by Grade 

In [35]:
#creating series for reading scores by grade
ninth_grade_reading = school_complete_df.loc[is_ninth_grade,:].groupby("school_name")["reading_score"].mean()
tenth_grade_reading = school_complete_df.loc[is_tenth_grade,:].groupby("school_name")["reading_score"].mean()
eleventh_grade_reading = school_complete_df.loc[is_eleventh_grade,:].groupby("school_name")["reading_score"].mean()
twelfth_grade_reading = school_complete_df.loc[is_twelfth_grade,:].groupby("school_name")["reading_score"].mean()

In [36]:
grade_reading_summary_df = pd.DataFrame({
    "9th Grade": ninth_grade_reading,
    "10th Grade": tenth_grade_reading,
    "11th Grade": eleventh_grade_reading,
    "12th Grade": twelfth_grade_reading
})

grade_reading_summary_df.index.rename("School Names", inplace=True)


grade_reading_summary_df

,9th Grade,10th Grade,11th Grade,12th Grade
School Names,,,,
Bailey High School,81.303155,80.907183,80.907183,80.907183
Cabrera High School,83.676136,84.253219,84.253219,84.253219
Figueroa High School,81.198598,81.408912,81.408912,81.408912
Ford High School,80.632653,81.262712,81.262712,81.262712
Griffin High School,83.369193,83.706897,83.706897,83.706897
Hernandez High School,80.866860,80.660147,80.660147,80.660147
Holden High School,83.677165,83.324561,83.324561,83.324561
Huang High School,81.290284,81.512386,81.512386,81.512386
Johnson High School,81.260714,80.773431,80.773431,80.773431


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [62]:
# creating bins and labels for per student spending

spending_bins = [0, 585, 630, 644, 675]
bin_labels = ["<$584", "$585-$629", "$630-$644", "$645-$675"]
assert len(spending_bins) == len(bin_labels) + 1

In [63]:
# pulling needed columns from school summary df
scores_by_school_df = school_summary_df.loc[
    :,
    ["Average Math Score Per School", "Average Reading Score Per School", "% Math Passing Per School", "% Reading Passing Per School", "% Overall Passing Per School"]
]

In [64]:
scores_by_school_df["Budget By Student Groups"] = pd.cut(
    x=school_summary_df["Per Student Budget"], 
    bins=spending_bins, 
    labels=bin_labels
)

scores_by_school_df.groupby("Budget By Student Groups").mean()


,Average Math Score Per School,Average Reading Score Per School,% Math Passing Per School,% Reading Passing Per School,% Overall Passing Per School
Budget By Student Groups,,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-$629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-$644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-$675,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [59]:
# setting bins and labels: less than 1000, 1000-2000, 2000-5000

size_bins = [0, 1000, 2000, 5000]
size_bin_labels = ["<1000", "1000-2000", "2000-5000"]
assert len(size_bins) == len(size_bin_labels) + 1

In [60]:
scores_by_school_size_df = school_summary_df.loc[
    :,
    ["Average Math Score Per School", "Average Reading Score Per School", "% Math Passing Per School", "% Reading Passing Per School", "% Overall Passing Per School"]
]

In [61]:
# to run this, the formatting from the school summary df had to be changed to an object, otherwise an object can't be binned using ints
scores_by_school_size_df["School Size Groups"] = pd.cut(
    x=school_summary_df["Total Students Per School"],
    bins=size_bins,
    labels=size_bin_labels)

scores_by_school_size_df.groupby("School Size Groups").mean()


,Average Math Score Per School,Average Reading Score Per School,% Math Passing Per School,% Reading Passing Per School,% Overall Passing Per School
School Size Groups,,,,,
<1000,83.821598,83.929843,93.550225,96.099437,89.883853
1000-2000,83.374684,83.864438,93.599695,96.790680,90.621535
2000-5000,77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [54]:
scores_by_school_size_df = school_summary_df.loc[
    :,
    ["School Type", "Average Math Score Per School", "Average Reading Score Per School", "% Math Passing Per School", "% Reading Passing Per School", "% Overall Passing Per School"]
]

In [55]:
scores_by_school_size_df.groupby("School Type").mean()

,Average Math Score Per School,Average Reading Score Per School,% Math Passing Per School,% Reading Passing Per School,% Overall Passing Per School
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


### Observations

1. On average, charter schools tend to perform better in both Math and Reading in raw scores and in percentage passing by a significant amount. 

2. Overall,  a higher per student budget does not necessarily mean Math and Reading scores will be higher. In fact, the data seems to suggest that schools with lower per student budget tend to have higher Math and Reading test scores.